In [2]:
import torch
from transformers import pipeline

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
gpt2_pipeline = pipeline("text-generation", model="gpt2", device=device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda


In [5]:
outputs = gpt2_pipeline("Hello, My Name is Hasnaat", max_length=30, do_sample=True, truncation=True)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=30) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Hello, My Name is Hasnaat!

I am Hasnaat. I have a special place in your heart. I serve you in secret, not for the sake of fame or fame, but because you would be my favorite, for that is what I am.

The same is true for you. I would like to give you this gift:

Your memory will never be the same again. You will live a future with me, but I will not let you forget your past.

There is no place where you will not be.

It is always with you.

I am your memory.

You will never forget me.

No, I am not going to forget you.

I want to be your memory.

You will never forget me.

No, I am not going to forget you.

You will never forget me.

You will never forget me.

I am your memory.

You will never forget me.

We are coming to an end.

You will never forget me.

I have been told that you will never forget me. I will not.

You will never forget me.

The moment you die, I will remember you.


In [6]:
type(gpt2_pipeline.tokenizer)

transformers.models.gpt2.tokenization_gpt2_fast.GPT2TokenizerFast

In [7]:
f"{gpt2_pipeline.model.num_parameters():,}"

'124,439,808'

In [8]:
input_string = "I Love NLP"
output = gpt2_pipeline(input_string)
print(output[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'generated_text': "I Love NLP, and also I love the LOVED the LOVED the LOVED this week. I love the LOVED the LOVED the LOVED this week.\n\nI'm not a professional speaker, and so I can't provide advice for you. But I can tell you that I get a lot of questions from people who are so busy asking me this question. (Laughter.) I know that I get a lot of questions from people who are so busy asking me this question. (Laughter.) I know that I get a lot of questions from people who are so busy asking me this question. I can tell you I get a lot of questions from people who are so busy asking me this question. I can tell you that I get a lot of questions from people who are so busy asking me this question. (Laughter.)\n\nNow, I had a lot of conversations with my colleagues over the last couple of weeks and I know that they're very concerned about my ability to explain how this could be a problem. So I'm gonna tell you a little bit about what I think is happening to our communication. (Laughter

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)
# Added EOS token as PAD Token to stop warning

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

input_string = "i love NLP"

In [10]:
# Now Passing the input string to tokenizer to get a list back of token ids as well as attention mask for the transformer

In [11]:
model_inputs = tokenizer(input_string, return_tensors='pt').to(device)

print(model_inputs)

{'input_ids': tensor([[   72,  1842,   399, 19930]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1]], device='cuda:0')}


In [12]:
# generate the output token ids

output = model.generate(**model_inputs, num_beams=10, early_stopping=True)


print(output)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[   72,  1842,   399, 19930,   355,   881,   355,   314,   466,    11,
           475,   314,   836,   470,   760,   611,   314,  1183,  1683,   307,
          1498,   284,   466,   340]], device='cuda:0')


In [13]:
output_string = tokenizer.decode(output[0])
print(output_string)

i love NLP as much as I do, but I don't know if I'll ever be able to do it


In [14]:
# Sampling Strategies

In [15]:
# Higher the Temperature the more diverse the text generation will be
# , the lower the temperature the less diverse and more deterministic it'll be
# 0 temp == same output for the input

In [16]:
# importing dataset

In [17]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="train.json")

Generating train split: 0 examples [00:00, ? examples/s]

In [18]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 14548
    })
})

In [19]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)


In [20]:
tokenizer.pad_token = tokenizer.eos_token

def tokenize_func(batch):
    inputs = tokenizer(
        batch["input_text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    labels = tokenizer(
        batch["target_text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_dataset = dataset.map(tokenize_func, batched=True, remove_columns=dataset["train"].column_names)


Map:   0%|          | 0/14548 [00:00<?, ? examples/s]

In [21]:
for i in range(0,5):
  print(tokenized_dataset['train'][i])

{'input_ids': [47, 325, 12003, 12, 8189, 25, 2163, 651, 62, 5420, 351, 890, 890, 4578, 257, 326, 5860, 890, 890, 2251, 890, 890, 9093, 329, 1312, 422, 352, 284, 1802, 10535, 830, 19889, 48816, 1312, 416, 838, 319, 1123, 24415, 611, 257, 18189, 1312, 14267, 262, 1334, 286, 262, 9052, 2073, 900, 9093, 284, 1312, 1220, 362, 2270, 220, 220, 1441, 9093, 220, 14580, 318, 257, 890, 890, 2163, 351, 890, 890, 4578, 257, 2251, 18253, 610, 456, 321, 13627, 890, 890, 1006, 329, 1312, 796, 352, 284, 1802, 10535, 830, 19889, 48816, 1312, 416, 838, 319, 1123, 24415, 611, 257, 18189, 1312, 467, 284, 262, 923, 286, 262, 9052, 2073, 8333, 1312, 532, 352, 284, 1006, 2245, 262, 9052, 220, 220, 1441, 357, 5420, 532, 257, 8, 1635, 257, 220, 220, 13627, 890, 890, 9633, 1312, 11, 256, 11, 285, 11, 299, 11, 474, 290, 479, 13627, 890, 890, 82, 300, 290, 374, 13627, 890, 890, 3509, 87, 796, 657, 1100, 300, 290, 374, 13627, 890, 890, 7885, 1006, 6784, 796, 651, 62, 5420, 7, 81, 8, 611, 374, 318, 1342, 621, 1006, 

In [22]:
# Fine Tuning the model

In [23]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.9 MB/s eta 0:00:00


In [24]:
from transformers import TrainingArguments, Trainer
import evaluate

In [38]:
training_args = TrainingArguments(
    output_dir="code-generation-gpt2",
    num_train_epochs=2,
    eval_strategy="epoch"
)


In [39]:
metric = evaluate.load("accuracy")

In [31]:
def compute_metric(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [40]:
from transformers import DataCollatorForLanguageModeling

In [41]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [42]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["train"],
    compute_metrics=compute_metric,
    data_collator=data_collator
)

In [43]:
trainer.train()

Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 5.37 GiB. GPU 0 has a total capacity of 14.74 GiB of which 2.53 GiB is free. Process 123218 has 12.21 GiB memory in use. Of the allocated memory 8.22 GiB is allocated by PyTorch, and 3.86 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [46]:
trainer.save_model()

In [47]:
!ls code-generation-gpt2

checkpoint-1000  generation_config.json  special_tokens_map.json  vocab.json
checkpoint-1500  merges.txt		 tokenizer_config.json
checkpoint-500	 model.safetensors	 tokenizer.json
config.json	 runs			 training_args.bin


In [48]:
!du -h code-generation-gpt2/model.safetensors

475M	code-generation-gpt2/model.safetensors


In [ ]:
# Testing Fine Tuned Model

In [49]:
from transformers import pipeline
from IPython.display import Markdown

input_text = "read integer n from input, initialize variable sum = 0, for i from 1 to n, add i to sum, print sum"
code_gen = pipeline('text-generation', model='./code-generation-gpt2', tokenizer=tokenizer,max_new_tokens=200,temperature=0.7, do_sample=True)
output = (code_gen(input_text)[0])['generated_text']
print(output)

Device set to use cuda:0


read integer n from input, initialize variable sum = 0, for i from 1 to n, add i to sum, print sum and newline 
Python code:  let n be a integer let a, b, c be long longs read n for i = 1 to n for j = 1 to n if i is odd print i    print "YES" otherwise print "NO" 
Python code:  let n be a integer let a, b, c be long longs read n for i = 1 to n for j = 1 to n if i is odd print i   print "YES" else print "NO"  
Python code:  let n be a integer let a, b, c be long longs read n for i = 1 to n for j = 1 to n if i is odd print i   print "YES" else print "NO" 
Python code:  let n be a integer let a, b, c be long longs read n for i = 1 to n for j = 1 to n if i is odd


In [ ]:
# we can replace the pred and references w actual values to compute bleu score

In [51]:
from evaluate import load

bleu = load("bleu")
results = bleu.compute(predictions=["print(sum(range(1, n+1)))"],
                       references=["print(sum(range(1, n+1)))"])
print(results)


{'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 14, 'reference_length': 14}
